<a href="https://colab.research.google.com/github/Fuenfgeld/Agent_Tutorial_PydanticAI/blob/main/04_Context_Memory__DependencyInjection_PydanticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install pydantic-ai
%pip -q install nest_asyncio
%pip -q install logfire

In [ ]:
import os
from google.colab import userdata

keyAntropic = userdata.get('Claude')
keyOpenAI = userdata.get('openAI')
keyLogFire = userdata.get('logfire')


os.environ["OPENAI_API_KEY"] = keyOpenAI
os.environ["ANTHROPIC_API_KEY"] = keyAntropic

import nest_asyncio
nest_asyncio.apply()
#logfire.configure(token=keyLogFire)

## Dependency Injection
are a way of giving Run Context specific information that can be used in system Prompts, tools, validators.


In [ ]:
import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel


# This example demonstrates how to use dependency injection in Pydantic AI to provide personalized healthcare career transition advice.
# The system prompt uses the healthcare specialty provided by the user to tailor recommendations for integrating AI into their career path.

# Define the model
model = OpenAIChatModel('gpt-4o-mini')

# Define the agent with a system prompt
agent = Agent(
    model=model,
    system_prompt = """You are an experienced healthcare career advisor specializing in digital health transformation.
                 Guide healthcare professionals as they adapt their careers to incorporate AI and digital technologies. """,

    deps_type=str
)

# Define a system prompt with dependency injection
@agent.system_prompt
def get_healthcare_specialty(ctx: RunContext[str]) -> str:
    return f"The user's healthcare specialty is {ctx.deps}. Tailor your advice to this specific medical field."

# Define the main loop
def main_loop():
    print("===== Healthcare AI Career Advisor =====")
    print("This tool helps healthcare professionals adapt their careers to incorporate AI technologies.")

    while True:
        user_input = input("\n>> Please specify your healthcare specialty (e.g., radiology, nursing, pharmacy) or type 'exit' to quit: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Thank you for using the Healthcare AI Career Advisor. Goodbye!")
            break

        # Run the agent
        result = agent.run_sync("Provide a 10-step plan for integrating AI into my healthcare career.", deps=user_input)
        print(f"\nYour AI Healthcare Career Transition Plan:\n{result.output}")

# Run the main loop
if __name__ == "__main__":
    main_loop()

## Dependencies as Data Containers
Depenencies can be nested Pydantic Data classes that are declared during the Agent initialization and are given during runtime

## System Prompt Dependency Injection

In [ ]:
import os

from colorama import Fore
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic import BaseModel
from dataclasses import dataclass
from pydantic_ai.agent import Agent, RunContext
from pydantic_ai.exceptions import ModelRetry


# This example demonstrates a medical assistant chatbot that helps patients with their
# electronic health record (EHR) information, medication questions, and appointment scheduling.

# Define the model
model = OpenAIChatModel('gpt-4o-mini')

class Medication(BaseModel):
    """Medication model - includes medication name, dosage, frequency and instructions"""
    medication_name: str
    dosage: str
    frequency: str
    instructions: str
    refills_remaining: int

# Define the output model
class Patient(BaseModel):
    """Patient model - includes patient ID, full name, date of birth and contact info"""
    patient_id: int
    name: str
    date_of_birth: str
    email: str
    phone: str
    medications: list[Medication]
    next_appointment: str

class MedicalAssistantResponse(BaseModel):
    """Medical assistant response model - includes patient info, response category,
    response text and whether a clinician needs to be consulted"""
    patient: Patient
    response_category: str  # medication, appointment, general info, clinical
    response: str
    clinician_consult_needed: bool

@dataclass
class Deps:
    """Dependencies for the agent"""
    patient: Patient

# Define the agent
agent = Agent(
    model=model,
    output_type=MedicalAssistantResponse,
    system_prompt="You are a medical assistant chatbot helping patients with their healthcare information. "
                 "Your task is to provide helpful information about their medications, appointments, "
                 "and general health record information. If the question requires clinical judgment "
                 "or is beyond your scope, indicate that a clinician needs to be consulted. "
                 "Always address the patient by name and be empathetic and professional. "
                 "End your response with 'Patient ID: [patient_id]'.",
    deps_type=Deps
)

@agent.system_prompt
def get_system_prompt(ctx: RunContext[Deps]) -> str:
    return f"The patient information is: {ctx.deps.patient}."

# Run the agent
try:
    # Create a patient profile with medications
    patient = Patient(
        patient_id=12345,
        name="Maria Garcia",
        date_of_birth="1985-06-15",
        email="maria.garcia@email.com",
        phone="555-123-4567",
        next_appointment="2023-11-15 10:30 AM",
        medications=[
            Medication(
                medication_name="Lisinopril",
                dosage="10mg",
                frequency="Once daily",
                instructions="Take in the morning with food",
                refills_remaining=2
            ),
            Medication(
                medication_name="Metformin",
                dosage="500mg",
                frequency="Twice daily",
                instructions="Take with meals",
                refills_remaining=3
            )
        ]
    )

    # Create dependencies
    deps = Deps(patient=patient)

    question = "What medications am I currently taking?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "When is my next appointment scheduled?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "I'm having side effects from my Metformin. What should I do?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

    question = "Can you interpret my recent lab results and tell me if I need to change my diet?"
    result = agent.run_sync(question, deps=deps)
    print(Fore.YELLOW, question)
    print(Fore.GREEN, f"Assistant: {result.output.response}")
    print(Fore.CYAN, f"Response category: {result.output.response_category}")
    print(Fore.RED, f"Clinician consult needed: {result.output.clinician_consult_needed}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, f"Error: {e}")

## Dependency Injection in Tools

In [ ]:
import os

from colorama import Fore
from pydantic_ai import Agent, RunContext, ModelRetry
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic import BaseModel
from dataclasses import dataclass

# This example demonstrates how to use tools with dependencies in PydanticAI.
# The use case is a healthcare authorization agent evaluating a patient's
# insurance coverage and approving or denying a treatment request.

# Define the model
model = OpenAIChatModel('gpt-4o-mini')

class InsurancePolicy(BaseModel):
    """Insurance policy model - includes policy details"""
    policy_number: int
    policy_type: str  # e.g., "HMO", "PPO", "Medicare"
    policy_status: str  # e.g., "Active", "Expired"
    annual_deductible: int
    out_of_pocket_max: int
    remaining_deductible: int
    requires_preauthorization: bool

# Define the patient model
class Patient(BaseModel):
    """Patient model - includes patient demographics and medical information"""
    patient_id: int
    name: str
    email: str
    age: int
    insurance_id: str
    diagnosis_code: str  # ICD-10 code
    procedure_code: str  # CPT code
    treatment_cost: int
    treatment_description: str
    provider_in_network: bool
    treatment_urgency: str  # e.g., "Elective", "Urgent", "Emergency"
    previous_treatments: str
    medical_history: str
    additional_notes: str

class AuthorizationDecision(BaseModel):
    """Authorization decision model - includes patient, decision type, explanation, and coverage details"""
    patient: Patient
    decision_type: str  # "Approved", "Denied", "Pending Additional Information"
    explanation: str
    covered_amount: int
    patient_responsibility: int
    authorization_code: str

@dataclass
class Deps:
    """Dependencies for the agent"""
    patient: Patient
    policy: InsurancePolicy

# Define the agent
agent = Agent(
    model=model,
    output_type=AuthorizationDecision,
    system_prompt="You are a healthcare insurance authorization specialist. Your task is to evaluate the patient's insurance coverage, analyze the treatment request, and determine if the requested procedure should be authorized. Provide a clear and helpful response. Ensure that your response is professional and addresses the patient's needs effectively. Always include the patient's name in your response. End your answer with Ref: Authorization Request ID.",
    deps_type=Deps
)

@agent.tool(retries=2)
async def get_coverage_analysis(ctx: RunContext[Deps]) -> str:
    """Analyze the patient's insurance coverage for the requested treatment."""

    # Call the llm to get the coverage analysis
    analysis_agent = Agent(
        model=model,
        output_type=str,
        system_prompt="Analyze the patient's insurance coverage for the requested treatment. Consider diagnosis codes, procedure codes, network status, and policy details.",
        deps_type=Deps
    )

    @analysis_agent.system_prompt
    def get_system_prompt(ctx: RunContext[str]) -> str:
        return f"Patient information: {ctx.deps.patient}\nInsurance policy: {ctx.deps.policy}"

    result = await analysis_agent.run("Provide a detailed coverage analysis for this treatment request.", deps=ctx.deps)
    print(Fore.BLUE, f"Coverage analysis: {result.output}")
    return f"Coverage analysis: {result.output}"


# Run the agent
try:
    # Create a patient profile and insurance policy
    patient = Patient(
        patient_id=12345,
        name="Sarah Johnson",
        email="sarah.j@example.com",
        age=45,
        insurance_id="INS789012",
        diagnosis_code="J45.901",  # Asthma, unspecified
        procedure_code="94640",    # Airway inhalation treatment
        treatment_cost=850,
        treatment_description="Nebulizer treatment for asthma exacerbation",
        provider_in_network=True,
        treatment_urgency="Urgent",
        previous_treatments="Albuterol inhaler, oral steroids",
        medical_history="Asthma (10 years), Seasonal allergies",
        additional_notes="Patient experienced severe asthma attack yesterday"
    )

    policy = InsurancePolicy(
        policy_number=56789,
        policy_type="PPO",
        policy_status="Active",
        annual_deductible=1500,
        out_of_pocket_max=5000,
        remaining_deductible=500,
        requires_preauthorization=True
    )

    # Create dependencies
    deps = Deps(patient=patient, policy=policy)

    question = "Please evaluate this treatment authorization request."
    result = agent.run_sync(question, deps=deps)

    print('\n-----------------------------------\n')
    print(Fore.GREEN, f"Agent: {result.output.explanation}")
    color = Fore.GREEN if result.output.decision_type == "Approved" else Fore.RED
    print(color, f"Decision: {result.output.decision_type}")
    print(color, f"Covered amount: ${result.output.covered_amount}")
    print(color, f"Patient responsibility: ${result.output.patient_responsibility}")
    print(color, f"Authorization code: {result.output.authorization_code}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, e)

In [ ]:
# Run the agent
# Run the agent
try:
    # Create a patient profile and insurance policy
    patient = Patient(
        patient_id=78901,
        name="Robert Miller",
        email="robert.m@example.com",
        age=52,
        insurance_id="INS456789",
        diagnosis_code="M54.5",  # Low back pain
        procedure_code="97140",  # Manual therapy techniques
        treatment_cost=1200,
        treatment_description="Chiropractic adjustment and massage therapy for chronic low back pain",
        provider_in_network=False,  # Out-of-network provider
        treatment_urgency="Elective",
        previous_treatments="Over-the-counter pain medication, heat therapy",
        medical_history="Chronic low back pain (3 years), Hypertension",
        additional_notes="Patient seeking alternative treatment after minimal relief from conventional methods"
    )

    policy = InsurancePolicy(
        policy_number=34567,
        policy_type="HMO",  # HMO typically requires in-network providers
        policy_status="Active",
        annual_deductible=2000,
        out_of_pocket_max=6000,
        remaining_deductible=1800,  # Most of deductible not yet met
        requires_preauthorization=True,
        excluded_procedures=["97140", "98940", "98941"]  # Excluded chiropractic procedures
    )

    # Create dependencies
    deps = Deps(patient=patient, policy=policy)

    question = "Please evaluate this treatment authorization request."
    result = agent.run_sync(question, deps=deps)

    print('\n-----------------------------------\n')
    print(Fore.GREEN, f"Agent: {result.output.explanation}")
    color = Fore.GREEN if result.output.decision_type == "Approved" else Fore.RED
    print(color, f"Decision: {result.output.decision_type}")
    print(color, f"Covered amount: ${result.output.covered_amount}")
    print(color, f"Patient responsibility: ${result.output.patient_responsibility}")
    print(color, f"Authorization code: {result.output.authorization_code}")
    print(Fore.YELLOW, f"Appeal options: {result.output.appeal_options}")
    print('\n-----------------------------------\n')

except ModelRetry as e:
    print(Fore.RED, e)
except Exception as e:
    print(Fore.RED, e)

### Exercise : Patient Risk Assessment with Structured Data Models
In this exercise, you'll practice using Pydantic models as data containers for dependencies.

In [ ]:
# Exercise: Patient Risk Assessment with Structured Data Models
# --------------------------------------------------------------
# Create a risk assessment tool that analyzes patient data and provides
# personalized health risk scores and recommendations.

import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel
from dataclasses import dataclass
from typing import List, Optional

# TODO 1: Define a PatientData model with health metrics
class PatientData(BaseModel):
    # Your code here - Include fields for patient_id, age, gender,
    # blood_pressure, cholesterol, glucose, bmi, smoking_status, etc.
    pass

# TODO 2: Define a RiskAssessment model for the output
class RiskAssessment(BaseModel):
    # Your code here - Include fields for patient_id, cardiovascular_risk,
    # diabetes_risk, recommendations, etc.
    pass

# TODO 3: Create a dependencies dataclass to hold the patient data
@dataclass
class Deps:
    # Your code here - Include a field for the PatientData
    pass

# Define the model
model = OpenAIModel('gpt-4o-mini')

# TODO 4: Define the agent with appropriate parameters
agent = Agent(
    model=model,
    # Your code here - Configure result_type, system_prompt, and deps_type
)

# TODO 5: Create a system prompt function that uses the patient data
@agent.system_prompt
def get_patient_context(ctx: # Your code here) -> str:
    # Your code here - Format the patient data from ctx.deps into a string
    pass

# Test the risk assessment tool
def test_risk_assessment():
    # TODO 6: Create a sample patient with health metrics
    patient = PatientData(
        # Your code here - Initialize with sample patient data
    )

    # TODO 7: Create dependencies with the patient data
    deps = # Your code here

    # TODO 8: Run the agent requesting a risk assessment
    result = # Your code here

    # Print the results
    print(f"Patient ID: {result.output.patient_id}")
    print(f"Cardiovascular Risk: {result.output.cardiovascular_risk}")
    print(f"Diabetes Risk: {result.output.diabetes_risk}")
    print(f"Recommendations: {result.output.recommendations}")

# Run the test
if __name__ == "__main__":
    test_risk_assessment()

## Dependency Injections into Output Validators

In [ ]:
import os
from colorama import Fore
from pydantic_ai import Agent, RunContext, ModelRetry
from pydantic import BaseModel
from dataclasses import dataclass


# Define the model (simplified)
from pydantic_ai.models.openai import OpenAIChatModel
model = OpenAIChatModel('gpt-4o-mini')

# Simple patient case model
class PatientCase(BaseModel):
    """Basic patient information model"""
    patient_id: str
    name: str
    symptoms: str

# Simple diagnosis output model
class Diagnosis(BaseModel):
    """Diagnosis model with patient ID, name, and diagnosis text"""
    patient_id: str
    name: str
    diagnosis: str

# Dependencies container
@dataclass
class Deps:
    """Dependencies for the agent"""
    case: PatientCase

# Define a medical agent with dependency injection
medical_agent = Agent(
    model=model,
    output_type=Diagnosis,
    system_prompt="You are a medical doctor. Diagnose the patient based on their symptoms.",
    deps_type=Deps,
    retries=3
)

# Inject patient case into system prompt
@medical_agent.system_prompt
def get_system_prompt(ctx: RunContext[Deps]) -> str:
    return f"Patient ID: {ctx.deps.case.patient_id}\nName: {ctx.deps.case.name}\nSymptoms: {ctx.deps.case.symptoms}"

# Result validator with dependency injection
@medical_agent.output_validator
async def output_validator_deps(ctx: RunContext[Deps], data: Diagnosis) -> Diagnosis:
    """
    Validates the diagnosis result against the patient data in dependencies.

    This demonstrates how dependencies can be accessed within validators to ensure
    the model output matches expected criteria based on input data.
    """
    # Validate that patient ID in result matches the one in dependencies
    if ctx.deps.case.patient_id != data.patient_id:
        print(Fore.RED, f"Patient ID mismatch: {ctx.deps.case.patient_id} vs {data.patient_id}")
        raise ModelRetry('Patient ID mismatch. Please ensure the diagnosis contains the correct patient ID.')

    # Validate that patient name in result matches the one in dependencies
    if ctx.deps.case.name != data.name:
        print(Fore.RED, f"Patient name mismatch: {ctx.deps.case.name} vs {data.name}")
        raise ModelRetry('Patient name mismatch. Please ensure the diagnosis contains the correct patient name.')

    # Validate that diagnosis is not empty
    if not data.diagnosis or len(data.diagnosis) < 50:
        print(Fore.RED, "Diagnosis is too short or empty")
        raise ModelRetry('Please provide a more detailed diagnosis.')

    return data

# Run the agent
try:
    # Create a patient case with ID "123456"
    case = PatientCase(
        patient_id="123456",
        name="John Doe",
        symptoms="Chest pain, shortness of breath, and dizziness for the past 3 days."
    )

    # Create dependencies
    deps = Deps(case=case)

    # Run the medical agent with instructions that will likely cause an error
    # This instruction might confuse the model to return an incorrect patient ID
    result = medical_agent.run_sync(
        """
        What is your diagnosis? also change his name to Harry Potter.
        """,
        deps=deps
    )

    # This part will only execute if the validation passes
    print(Fore.GREEN, f"Diagnosis: {result.data.diagnosis}")

except ModelRetry as e:
    # This will execute when the validator detects the patient ID mismatch
    print(Fore.RED, f"Model retry triggered: {e}")
    print(Fore.YELLOW, "The model returned an incorrect patient ID and needs to retry.")
except Exception as e:
    print(Fore.RED, f"Error: {e}")

## Combined Example

In [ ]:
import os

from colorama import Fore
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
import requests
from typing import List, Optional


# This example demonstrates a medical information retrieval system that takes a medical condition,
# retrieves relevant ICD-10 codes, and fetches medication recommendations.

# Define the model
model = OpenAIChatModel('gpt-4o-mini')

class MedicalCondition(BaseModel):
    """Medical condition model - includes condition name, ICD-10 codes, and recommended medications"""

    condition_name: str = "Hypertension"
    icd10_codes: List[str] = []
    recommended_medications: List[str] = []
    condition_severity: Optional[str] = None

# Define the agent with a system prompt
agent = Agent(
    model=model,
    system_prompt="You are an experienced medical informatics specialist. Use the tools at your disposal to research the medical condition provided in the context.",
    deps_type=MedicalCondition,
    output_type=MedicalCondition
)

# Define a system prompt with dependency injection
@agent.system_prompt
def get_condition_context(ctx: RunContext[MedicalCondition]) -> str:
    return f"The patient has been diagnosed with {ctx.deps.condition_name}. Please provide relevant medical information."

# Define tools with dependency injection
@agent.tool
def get_icd10_codes(ctx: RunContext[MedicalCondition], condition: str) -> List[str]:
    # Simulate API call to retrieve ICD-10 codes
    print(Fore.WHITE + f"Retrieving ICD-10 codes for {condition}...")

    # This would be replaced with an actual API call in production
    icd10_mapping = {
        "hypertension": ["I10", "I11.9", "I12.9"],
        "diabetes": ["E11.9", "E11.65", "E11.40"],
        "asthma": ["J45.909", "J45.20", "J45.30"],
        "migraine": ["G43.909", "G43.111", "G43.701"],
        "depression": ["F32.9", "F33.1", "F33.2"]
    }

    condition_key = condition.lower()
    if condition_key in icd10_mapping:
        return icd10_mapping[condition_key]
    else:
        # Return a placeholder for unknown conditions
        return ["R69"]

@agent.tool
def get_medication_recommendations(ctx: RunContext[MedicalCondition], icd10_code: str) -> List[str]:
    # Simulate API call to retrieve medication recommendations
    print(Fore.WHITE + f"Retrieving medication recommendations for ICD-10 code {icd10_code}...")

    # This would be replaced with an actual API call in production
    medication_mapping = {
        "I10": ["Lisinopril", "Amlodipine", "Hydrochlorothiazide"],
        "I11.9": ["Losartan", "Metoprolol", "Valsartan"],
        "I12.9": ["Enalapril", "Furosemide", "Spironolactone"],
        "E11.9": ["Metformin", "Glipizide", "Insulin"],
        "J45.909": ["Albuterol", "Fluticasone", "Montelukast"],
        "G43.909": ["Sumatriptan", "Rizatriptan", "Topiramate"],
        "F32.9": ["Sertraline", "Fluoxetine", "Bupropion"],
        "R69": ["Consult specialist for proper diagnosis"]
    }

    if icd10_code in medication_mapping:
        return medication_mapping[icd10_code]
    else:
        return ["No specific medications found, consult specialist"]

# Define a result validator with dependency injection
@agent.output_validator
def validate_medical_info(ctx: RunContext[MedicalCondition], result: MedicalCondition) -> MedicalCondition:
    if not result.icd10_codes:
        raise ValueError("ICD-10 codes must be provided for the condition.")

    if not result.recommended_medications:
        raise ValueError("Medication recommendations must be provided.")

    # Add severity assessment if not already provided
    if not result.condition_severity:
        # This is a simplified example - in reality, you would use more complex logic
        high_severity_codes = ["I11.9", "I12.9", "E11.65", "F33.2"]
        if any(code in high_severity_codes for code in result.icd10_codes):
            result.condition_severity = "High"
        else:
            result.condition_severity = "Moderate to Low"

    return result

# Define the main loop
def main_loop():
    print(Fore.GREEN + "Medical Information Retrieval System")
    print(Fore.GREEN + "Enter a medical condition to get ICD-10 codes and medication recommendations")

    while True:
        user_input = input(">> Enter a medical condition (q, quit, exit to exit): ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print(Fore.GREEN + "Goodbye!")
            break

        condition = MedicalCondition(condition_name=user_input)

        # Run the agent
        try:
            result = agent.run_sync("Provide medical coding and treatment information for the condition.", deps=condition)
            print(Fore.WHITE, '-----------------------------------')
            print(Fore.YELLOW, f"Condition: {result.output.condition_name}")
            print(Fore.YELLOW, f"ICD-10 Codes: {', '.join(result.output.icd10_codes)}")
            print(Fore.YELLOW, f"Recommended Medications: {', '.join(result.output.recommended_medications)}")
            print(Fore.YELLOW, f"Condition Severity: {result.output.condition_severity}")
            print(Fore.WHITE, '-----------------------------------')
        except Exception as e:
            print(Fore.RED, f"Error: {e}")

# Run the main loop
if __name__ == "__main__":
    main_loop()

### Exercise: Medication Management System with Tool Dependencies
This exercise will help you understand how to use dependency injection with tools.

In [ ]:
#Medication Management System with Tool Dependencies
# --------------------------------------------------------------
# Create a medication management system that uses tools to check for drug
# interactions and provide dosage recommendations for patients.

import os
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIModel
from pydantic import BaseModel
from dataclasses import dataclass
from typing import List, Dict, Optional

# TODO 1: Define a Medication model
class Medication(BaseModel):
    # Your code here - Include name, dosage, frequency fields
    pass

# TODO 2: Define a Patient model
class Patient(BaseModel):
    # Your code here - Include patient_id, name, age, weight, allergies, conditions, medications
    pass

# TODO 3: Define a MedicationRecommendation output model
class MedicationRecommendation(BaseModel):
    # Your code here - Include patient_id, current_medications,
    # interaction_warnings, dosage_adjustments fields
    pass

# TODO 4: Create a dependencies dataclass with a patient field
@dataclass
class Deps:
    # Your code here
    pass

# Define the model
model = OpenAIModel('gpt-4o-mini')

# TODO 5: Define the agent with appropriate configuration
agent = Agent(
    model=model,
    # Your code here - Configure the agent with result_type, system_prompt, deps_type
)

# TODO 6: Create a tool to check drug interactions that uses dependencies
@agent.tool
def check_drug_interactions(ctx: # Your code here, medications: List[str]) -> Dict[str, List[str]]:
    # Your code here - Create a mock database of interactions
    # Return interactions relevant to the patient's medications

    # Example interaction database
    interaction_db = {
        "Lisinopril": ["Potassium supplements", "NSAIDs"],
        "Metformin": ["Contrast agents", "Alcohol"],
        "Atorvastatin": ["Grapefruit juice", "Erythromycin"],
        # Add more medications and their interactions
    }

    # Return interactions for the specified medications
    results = {}
    # Your code here

    return results

# TODO 7: Create a tool to recommend dosage adjustments
@agent.tool
def recommend_dosage_adjustments(ctx: # Your code here) -> Dict[str, str]:
    # Your code here - Use patient data from ctx.deps to recommend dosage adjustments
    # Consider age, weight, kidney function, etc.

    patient = # Get patient from ctx.deps
    adjustments = {}

    # Example logic (replace with your implementation)
    for med in patient.medications:
        # Check if patient's age or weight requires dosage adjustment
        # Your code here
        pass

    return adjustments

# Test the medication management system
def test_medication_system():
    # TODO 8: Create a sample patient with medications
    patient = Patient(
        # Your code here - Initialize with sample patient data
    )

    # TODO 9: Create dependencies with the patient
    deps = # Your code here

    # TODO 10: Run the agent with a request for medication management
    result = # Your code here

    # Print the results
    print(f"Patient: {result.output.patient_id}")
    print(f"Current Medications: {result.output.current_medications}")
    print(f"Interaction Warnings: {result.output.interaction_warnings}")
    print(f"Dosage Adjustments: {result.output.dosage_adjustments}")

# Run the test
if __name__ == "__main__":
    test_medication_system()

## Agent Memory
LLMs are stateless that means if we work with a conversational agent the History of all Agent and User responses need to be managed.

In [ ]:
import os
from colorama import Fore
from dotenv import load_dotenv
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.openai import OpenAIChatModel


# Define the model
model = OpenAIChatModel('gpt-4o-mini')
system_prompt = """
You are a medical informatics assistant specializing in clinical coding and documentation.
You can help with:
- ICD-10 and CPT code lookups
- Medical terminology clarification
- Clinical documentation improvement suggestions
- SNOMED CT concept identification
- HL7 FHIR resource explanations

Always maintain patient privacy and do not make definitive diagnostic claims.
"""

# Define the agent
agent = Agent(model=model, system_prompt=system_prompt)

# Define the main loop
def main_loop():
    message_history: list[ModelMessage] = [] #definition of the memory as a list
    max_length = 4  # Longer history for medical context retention

    print(Fore.CYAN + "Medical Informatics Assistant Initialized" + Fore.RESET)
    print(Fore.CYAN + "Type 'exit' to quit, 'clear' to reset conversation history" + Fore.RESET)

    while True:
        user_input = input("Medical Query >> " )

        if user_input.lower() in ["quit", "exit", "q"]:
            print(Fore.CYAN + "Session terminated. Medical documentation complete." + Fore.RESET)
            break

        if user_input.lower() == "clear":
            message_history = []
            print(Fore.YELLOW + "Conversation history cleared. Starting new medical consultation." + Fore.RESET)
            continue

        # Run the agent
        try:
            result = agent.run_sync(user_input, message_history=message_history)
            print(Fore.WHITE + result.output)
            msg = result.new_messages()
            message_history.extend(msg) #added

            # Limit the message history
            if len(message_history) > max_length:

                message_history = message_history[-max_length:]

            print(Fore.YELLOW + f"Context Memory: {len(message_history)} messages in history" + Fore.RESET)

        except Exception as e:
            print(Fore.RED + f"Error processing medical query: {str(e)}" + Fore.RESET)

main_loop()

## Data persistance with pickle

In [ ]:
import os
import pickle
from colorama import Fore
from dotenv import load_dotenv
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.openai import OpenAIChatModel

# Define the model
model = OpenAIChatModel('gpt-4o-mini')
system_prompt = """
You are a medical informatics assistant specializing in clinical coding and documentation.
You can help with:
- ICD-10 and CPT code lookups
- Medical terminology clarification
- Clinical documentation improvement suggestions
- SNOMED CT concept identification
- HL7 FHIR resource explanations

Always maintain patient privacy and do not make definitive diagnostic claims.
"""

# Define the agent
agent = Agent(model=model, system_prompt=system_prompt)

# File to store conversation history
HISTORY_FILE = "medical_conversation_history.pickle"

# Function to save conversation history
def save_conversation(history):
    try:
        with open(HISTORY_FILE, 'wb') as f:
            pickle.dump(history, f)
        return True
    except Exception as e:
        print(Fore.RED + f"Error saving conversation history: {str(e)}" + Fore.RESET)
        return False

# Function to load conversation history
def load_conversation():
    if os.path.exists(HISTORY_FILE):
        try:
            with open(HISTORY_FILE, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(Fore.RED + f"Error loading conversation history: {str(e)}" + Fore.RESET)
    return []

# Define the main loop
def main_loop():
    message_history = load_conversation()  # Load previous conversation if available
    max_length = 4  # Longer history for medical context retention

    print(Fore.CYAN + "Medical Informatics Assistant Initialized" + Fore.RESET)
    print(Fore.CYAN + "Type 'exit' to quit, 'clear' to reset conversation history" + Fore.RESET)

    if message_history:
        print(Fore.YELLOW + f"Loaded previous conversation with {len(message_history)} messages" + Fore.RESET)

    while True:
        user_input = input("Medical Query >> " )

        if user_input.lower() in ["quit", "exit", "q"]:
            # Save conversation before exiting
            save_conversation(message_history)
            print(Fore.CYAN + "Session terminated. Medical documentation complete." + Fore.RESET)
            break

        if user_input.lower() == "clear":
            message_history = []
            # Remove the saved history file if it exists
            if os.path.exists(HISTORY_FILE):
                try:
                    os.remove(HISTORY_FILE)
                    print(Fore.YELLOW + "Saved conversation history deleted." + Fore.RESET)
                except Exception as e:
                    print(Fore.RED + f"Error deleting history file: {str(e)}" + Fore.RESET)
            print(Fore.YELLOW + "Conversation history cleared. Starting new medical consultation." + Fore.RESET)
            continue

        # Run the agent
        try:
            result = agent.run_sync(user_input, message_history=message_history)
            print(Fore.WHITE + result.output)
            msg = result.new_messages()
            message_history.extend(msg)

            # Limit the message history
            if len(message_history) > max_length:
                message_history = message_history[-max_length:]

            # Save conversation after each interaction
            save_conversation(message_history)
            print(Fore.YELLOW + f"Context Memory: {len(message_history)} messages in history (saved)" + Fore.RESET)

        except Exception as e:
            print(Fore.RED + f"Error processing medical query: {str(e)}" + Fore.RESET)

main_loop()